In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-risavopmax2')

# Import necessary libraries
import os

# Define the region of interest (Sundarbans coastal area)
sundarbans = ee.Geometry.Polygon(
    [[[88.008, 21.495],
      [89.418, 21.495],
      [89.418, 22.505],
      [88.008, 22.505]]])

# Define the time range
start_year = 2014
end_year = 2018

# Create a directory in Google Drive to save images
folder_name = 'Sundarbans_Timelapse'
os.makedirs(folder_name, exist_ok=True)

def save_image(year, month):
    # Load a Landsat image for the given month and year
    start_date = f'{year}-{month:02d}-01'
    end_date = f'{year}-{month:02d}-28'  # Assuming 28 days for simplicity

    landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                .filterBounds(sundarbans) \
                .filterDate(start_date, end_date) \
                .select(['SR_B4', 'SR_B3', 'SR_B2']) \
                .median()

    # Clip the image to the region of interest
    clipped_image = landsat.clip(sundarbans)

    # Define visualization parameters
    vis_params = {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # RGB bands
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }

    # Export the image to Google Drive
    export_task = ee.batch.Export.image.toDrive(
        image=clipped_image,
        description=f'Sundarbans_{month:02d}_{year}',
        folder=folder_name,
        fileNamePrefix=f'Sundarbans_{month:02d}_{year}',
        scale=30,
        region=sundarbans.getInfo()['coordinates'],
        fileFormat='GEO_TIFF'
    )
    export_task.start()

# Loop through the years and months and save images
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        save_image(year, month)

print("Export tasks started. Check your Google Drive for the images.")


Export tasks started. Check your Google Drive for the images.


In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 35.4 MB/s eta 0:00:00


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
os.mkdir('/content/drive/MyDrive/Sundarbans_images')
for i in os.listdir('/content/drive/MyDrive/Sundarbans_Timelapse'):
  dir = os.path.join('/content/drive/MyDrive/Sundarbans_Timelapse',i)
  with rasterio.open(dir) as src:
    # Read the RGB bands (B4, B3, B2)
    red = src.read(1)  # Band 1 in the GEO_TIFF (SR_B4)
    green = src.read(2)  # Band 2 in the GEO_TIFF (SR_B3)
    blue = src.read(3)  # Band 3 in the GEO_TIFF (SR_B2)
    red = np.nan_to_num(red)
    green = np.nan_to_num(green)
    blue = np.nan_to_num(blue)

    # Stack the bands into a single image array
    rgb = np.dstack((red, green, blue))

    # Normalize the bands for display
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())
    name = str(i[:-4] + '.png')
    new_path=os.path.join('/content/drive/MyDrive/Sundarbans_images',name)
    plt.imsave(new_path,rgb)

    # # Plot the image
    # plt.figure(figsize=(10, 10))
    # plt.imshow(rgb)
    # plt.title('Sundarbans 2014 (RGB)')
    # plt.axis('off')
    # plt.show()
